In [ ]:
%load_ext autoreload
%autoreload 2

In [23]:
import torch
import transformers

from peft.tuners.lora import Linear
import torch.nn.functional as F
from peft.utils.other import transpose
from peft import PeftModel

import numpy as np

from blora import forward, StreamingPeftModel
import scipy.stats as stats
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output


Linear.forward = forward
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
loras = ["jondurbin/airoboros-7b-gpt4-1.2-peft", "trl-lib/llama-7b-se-rl-peft", "winddude/wizardLM-LlaMA-LoRA-7B"]

In [25]:
model_path = "/home/ubuntu/llama-weights/7B/llama-7b"

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = transformers.LlamaForCausalLM.from_pretrained(model_path)
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path)
tokenizer.add_special_tokens({'pad_token': 'PAD'})

Loading checkpoint shards: 100%|██████████| 33/33 [00:11<00:00,  2.80it/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [26]:
def load_loras(model, loras):
    # peft throws error if lora name contains a dot
    adapters = [lora.replace(".", "_") for lora in loras]
    lora_map = {lora: adapter for lora, adapter in zip(loras, adapters)}
    model = PeftModel.from_pretrained(model, loras[0], adapter_name=adapters[0])
    for lora, adapter in zip(loras[1:], adapters[1:]):
        model = PeftModel.from_pretrained(model.base_model.model, lora, adapter_name=adapter)
    return model, lora_map

model, lora_map = load_loras(model, loras)

In [27]:
prompts = [
    "Outline a five sentence short story where a character stumbles upon a secret room in their house that contains relics from their future.",
    "Write a 6 line dialogue between a character and a magical creature that only they can see.",
    "Describe a four sentence scene where a character discovers a hidden talent that changes their life forever.",
    "Sculpt a three verse poem about the feeling of walking through a lush, vibrant garden in full bloom.",
    "Develop an eight sentence short story about a character who can bring their dreams into reality, but only for a limited time.",
    "Create a six sentence scene where a character finds themselves in a world where emotions are visible as colors surrounding each person.",
    "Design an nine line dialogue between a character and a sentient cloud that follows them everywhere they go.",
    "Narrate a 10 sentence story about a character who can switch between different realities, but can't control when or where they will end up.",
    "Draft a three verse poem about the feeling of encountering a breathtaking view from a mountaintop.",
    "Write a four sentence scene where a character discovers they can rewind time, but only in 10-second increments.",
    "Capture a five sentence short story about a character who can communicate with nature, seeking help from plants and animals to solve a mystery.",
    "Portray an eight line dialogue between a character and a ghost who is unaware of their own death."
    ]

In [30]:
inputs = [(p, random.choice(loras)) for p in prompts]
batch = tokenizer(prompts, return_tensors="pt", padding=True)
attention_masks = batch["input_ids"].ne(tokenizer.pad_token_id).to(model.device)

for name, module in model.named_modules():
    module.batch_lora_ids = [lora_map[inp[1]] for inp in inputs]

In [ ]:
model.generate(
    input_ids=batch['input_ids'],
    attention_mask=batch['attention_mask'],
    max_length=200,
)
batch_decoded = tokenizer.batch_decode(torch.cat([out.reshape(-1, 1) for out in outputs], dim=1))

In [31]:
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 1
timings=np.zeros((repetitions,1))

outputs = []

with torch.no_grad():
    for rep in range(repetitions):
        starter.record()
        for out in model.generate(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            max_length=200,
        ):
            outputs.append(out)
            batch_decoded = tokenizer.batch_decode(torch.cat([out.reshape(-1, 1) for out in outputs], dim=1))
            clear_output(wait=True)
            print("\n\n".join([lora + ":\n" + decoded for lora, decoded in zip(loras, batch_decoded)]))
        ender.record()
        torch.cuda.synchronize()
        curr_time = starter.elapsed_time(ender)
        timings[rep] = curr_time

mean_syn = np.sum(timings) / repetitions
std_syn = np.std(timings)
print(mean_syn / 1000, std_syn / 1000)

AttributeError: 'generator' object has no attribute 'reshape'

In [ ]:
x = np.array([1, 2, 4, 8, 16, 32, 64])
y1 = np.array([14.13, 16.05, 22.43, 35.18, 62.25, 114.11, 219.20])
y2 = 14.13 * x

plt.plot(x, y1, 'b-')
plt.plot(x, y2, 'r--')

plt.xlabel('Batch Size')
plt.ylabel('Time (s)')

plt.legend(['Batched Lora', 'Sequential'], loc='upper left')
plt.title('Generating 200 tokens with Llama-7B using Batched Lora vs Sequential on A100-80gb')

In [ ]:
# get slope of y1
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y1)
print(slope)
print(intercept)

# which lib to import starts from

In [ ]:
9.7 + 3 * 3.3